In [237]:
import pickle
import json
import requests
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso

In [238]:
number_of_entries = 100
link = f"https://api.cryptobay.io/bay/cryptobaygetauctionlist?\
data=%7B%22object%22%3A%7B%7D%2C%22token_type%22%3A1%2C%22state%22%3A%220%22%2C%22\
order_by%22%3A%222%22%2C%22page_num%22%3A0%2C%22page_size%22%3A{number_of_entries}%7D"

r = requests.get(link)

raw_json = r.json()

In [239]:
def get_binance_price() -> int:
    """
    Example
            {
        "exchange": "binance",
        "pair": "bnbusdt",
        "price": 335.8,
        "ask": 335.8,
        "bid": 335.7
        }
    """
    price_json = requests.get("https://coinograph.io/ticker/?symbol=binance:bnbusdt").json()
    return price_json['price']


In [240]:
def get_ship_stats_selling(bnb_price: int = get_binance_price()):
    ship_id_list = pd.DataFrame([element['token_id'] for element in raw_json['data']['auctions']])
    ship_class_list = pd.DataFrame([element['class'] for element in raw_json['data']['auctions']])
    ship_price_list_BNB = pd.DataFrame([ round(int(element['buyout_price']) / 1000000000, 3) for element in raw_json['data']['auctions'] ])
    ship_price_list_USD = pd.DataFrame([ round(int(element['buyout_price']) / 1000000000 * bnb_price, 2) for element in raw_json['data']['auctions'] ])

    boat_space_list = pd.DataFrame([ element['raw_space'] for element in raw_json['data']['auctions']])
    boat_speed_list = pd.DataFrame([ element['raw_speed'] for element in raw_json['data']['auctions']])
    boat_skill_list = pd.DataFrame([ element['raw_skill'] for element in raw_json['data']['auctions']])
    boat_defence_list = pd.DataFrame([ element['raw_defence'] for element in raw_json['data']['auctions']])
    boat_attack_list = pd.DataFrame([ element['raw_attack'] for element in raw_json['data']['auctions']])
    boat_morale_list = pd.DataFrame([ element['raw_morale'] for element in raw_json['data']['auctions']])
    
    concat_DF = pd.concat([ship_id_list, ship_class_list, \
                        boat_space_list, boat_speed_list, boat_skill_list, \
                        boat_defence_list, boat_attack_list, boat_morale_list,\
                        ship_price_list_USD, ship_price_list_BNB], axis=1)
    
    concat_DF.columns
    
    return concat_DF

In [241]:
initial_bnb_price = 431
bnb_price= get_binance_price()

# IMPORT MODELS

model_USD = pickle.load(open('db/model_USD.sav', 'rb'))

zztest = np.array([5,13,15,6,11,5])
zztest = zztest.reshape(1, -1)

model_USD.predict(zztest)


array([61.35054854])

In [242]:
columns = ['transaction_id', 'ship_id', 'ship_class', 'ship_durability', 'ship_owner', \
        'ship_attr_space', 'ship_attr_speed', 'ship_attr_skill', 'ship_attr_defence', 'ship_attr_attack', 'ship_attr_morale', \
        'ship_parts_keel', 'ship_parts_sail', 'ship_parts_side', 'ship_parts_bow', 'ship_parts_cabin', 'ship_parts_stern', \
        'ship_sold_price']

In [243]:
#ship_price_list = [ round(int(element['buyout_price']) / 1000000000 * bnb_price, 2) for element in raw_json['data']['auctions']]

In [244]:
test_df = get_ship_stats_selling()
test_df.columns = ['ship_id', 'ship_class','ship_attr_space', \
    'ship_attr_speed', 'ship_attr_skill', 'ship_attr_defence', \
    'ship_attr_attack', 'ship_attr_morale', 'selling_price_USD', 'selling_price_BNB']
test_df.head(2)

,ship_id,ship_class,ship_attr_space,ship_attr_speed,ship_attr_skill,ship_attr_defence,ship_attr_attack,ship_attr_morale,selling_price_USD,selling_price_BNB
0,11745,0,12,9,15,15,9,7,192.80,0.5
1,16033,0,6,5,11,5,5,9,38.56,0.1


In [245]:
pred_columns = ['ship_attr_space', 'ship_attr_speed', 'ship_attr_skill', \
                'ship_attr_defence', 'ship_attr_attack', 'ship_attr_morale']

test_predict_USD = test_df[pred_columns]

test_predict_USD.head()

,ship_attr_space,ship_attr_speed,ship_attr_skill,ship_attr_defence,ship_attr_attack,ship_attr_morale
0,12,9,15,15,9,7
1,6,5,11,5,5,9
2,9,15,8,11,7,11
3,6,10,14,14,9,13
4,6,14,13,14,12,13


In [246]:
predicted_y = model_USD.predict(test_predict_USD)
predicted_y = pd.Series(predicted_y)
predicted_y = predicted_y.round(2)

In [247]:
_df = pd.concat([test_df, predicted_y], axis=1)
_df.rename(columns={0: 'predicted_price_USD'}, inplace=True)
_df['flip_ratio'] = _df['predicted_price_USD'] / _df['selling_price_USD']
_df['flip_ratio'] = _df['flip_ratio'].round(2)
gold_df = _df[['ship_id', 'selling_price_BNB', 'selling_price_USD', 'predicted_price_USD', 'flip_ratio']]

In [248]:
(gold_df[gold_df['flip_ratio'] > 1.5]\
    .sort_values('flip_ratio', ascending=False))

,ship_id,selling_price_BNB,selling_price_USD,predicted_price_USD,flip_ratio
81,519,0.12,46.27,83.62,1.81
3,17568,0.12,46.27,81.45,1.76
67,28018,0.11,42.42,68.03,1.60
96,11746,0.10,38.56,58.80,1.52
